In [5]:
import numpy as np
import math
import matplotlib as plt

In [4]:
def calc_auv2_accel(T, alpha, theta, mass=100):
    Thorz = T * math.sin(alpha)
    Tvert = T * math.cos(alpha)
    tothorizontal = Thorz[0] - Thorz[1] + Thorz[2] - Thorz[3]
    totvertical = Tvert[:2] - Tvert[2:]
    accel = np.array([tothorizontal, totvertical])
    rotate = np.array(
        [[math.cos(theta), -math.sin(theta)], [math.sin(theta), math.cos(theta)]]
    )
    return np.dot(rotate, accel) / mass


def calc_auv2_ang_accel(T, alpha, L, l, interia=100):
    Thorz = T * math.sin(alpha)
    Tvert = T * math.cos(alpha)
    torques = l * Thorz + L * Tvert
    return torques[0] - torques[1] + torques[2] - torques[3]

def sim_auv_motion(T, alpha, L, l, lmass, inertia, dt=0.1, t_final=10, x0=0, y0=0, theta=0):
    count = 0
    curang = theta
    timestamps = np.array([0])
    x = np.array([x0])
    y = np.array([y0])
    theta = np.array([theta])
    v = np.array([0, 0])
    omega = np.array([0])
    a = np.array([0, 0])
    while count <= t_final:
        count += dt
        timestamps.append(count)
        accel = calc_auv2_accel(T, alpha, curang)
        a.append(accel)
        angaccel = calc_auv2_ang_accel(T, alpha, L, l, inertia)
        newv = v[-1] + accel * dt
        newomega = omega[-1] + angaccel * dt
        v.append(newv)
        omega.append(newomega)
        newtheta = theta[-1] * dt + 0.5 * angaccel * dt * dt
        theta.append(newtheta)
        newcoords = np.array([x[-1], y[-1]]) * dt + 0.5 * accel * dt * dt
        x.append(newcoords[0])
        y.append(newcoords[1])
    return timestamps, x, y, theta, v, omega, a

In [6]:
t, x, y, theta, v, omega, a = sim_auv_motion(np.array([10,10,10,10]),  3, 3, 3, 3, 3)
coords = np.stack(x,y)
plt.plot(coords)

AttributeError: 'numpy.ndarray' object has no attribute 'append'